In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

%pip install -q spuco

In [3]:
%pip install -q pytorch-metric-learning

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from pytorch_metric_learning import losses
import torch.optim as optim
from torch.optim import lr_scheduler
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Config:
  spurious_correlation_strength  = 0.5
  spurious_feature_difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
  label_noise = 0.001
  size = 28
  flip_probability = 0.5
  batch_size = 256
  backbone_pretrained = False
  projection_dim = 128
  T_max = 10
  lr = 1e-4
  min_lr = 1e-5
  weight_decay = 1e-6
  temperature = 0.5

## Dataset

In [6]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="train",
    label_noise=Config.label_noise
)
trainset.initialize()

valset = SpuCoMNIST(
    root="/data/mnist/",

    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="test",
    label_noise=Config.label_noise
)
valset.initialize()


100%|██████████| 9912422/9912422 [00:00<00:00, 106272219.00it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 39025674.56it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 36399236.83it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3880735.13it/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 10000/10000 [00:01<00:00, 6625.34it/s]


In [7]:

## Augmentation
from torchvision.transforms import v2

size = 28
train_transform = transforms.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    v2.RandomGrayscale(p=0.2),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    v2.ToImage(),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [8]:
class SpucoDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img = self.dataset[index][0]
        label = self.dataset[index][1]

        if self.transform is None:
          return img, label

        augmented_img = self.transform(img)
        return (img, augmented_img), label

In [9]:
train_dataset = SpucoDataset(trainset, train_transform)
val_dataset = SpucoDataset(valset, test_transform)

dataloaders = {'train': DataLoader(dataset=train_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=True),
         'eval': DataLoader(dataset=val_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=False)}

In [10]:
from IPython.display import display
for (k, v) in valset.group_partition.items():
  print(f"Group: {k}, Qty: {len(v)}")
  img = T.ToPILImage()(valset[v[0]][0])
  display(img)

Group: (3, 4), Qty: 397


Group: (1, 1), Qty: 409


Group: (0, 0), Qty: 423


Group: (0, 4), Qty: 423


Group: (2, 2), Qty: 375


...

Group: (1, 4), Qty: 408


Group: (3, 1), Qty: 397


Group: (4, 0), Qty: 397


Group: (0, 1), Qty: 423


Group: (4, 2), Qty: 397


In [11]:
T.ToPILImage()(valset[v[0]][0])

## Contrastive Loss

In [12]:
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#ntxentloss
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#selfsupervisedloss
criterion = losses.NTXentLoss(temperature=Config.temperature)
criterion = losses.SelfSupervisedLoss(criterion)

## Model

In [13]:
## https://github.com/berfukaraca/SSL-Cifar10-ContrastiveLearning/blob/main/colab_notebooks/SSL_SimCLR_Cifar10_CL.ipynb

class SimCLR_model(nn.Module):
    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()
        self.n_features = n_features
        self.base_encoder = base_encoder

        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i):
        out = self.projectionHead(self.base_encoder(x_i))

        return out

In [14]:
## model and optimizer
model = models.resnet50()
# print(model)
clr_model = SimCLR_model(model, Config.projection_dim, 1000) ## output dimension of resnet18 is 1000
optimizer = optim.Adam(clr_model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
## cosine annealing lr: https://arxiv.org/pdf/1608.03983.pdf
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr)

clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [15]:
import os
def train(epoch):
    print('\nEpoch: %d' % epoch)
    clr_model.train()
    s=0
    tot_loss = 0
    total = 0
    batch_idx=0
    i=0
    batch_num =1
    val_loss = 0
    for batch_idx, (images, _) in enumerate(dataloaders['train']):
        batch_tot_loss = 0
        if (batch_num % 50 == 1):
          print("in batch", batch_num)
        batch_num=batch_num+1
        images, augmented_images = images[0], images[1]
        images, augmented_images = images.to(device), augmented_images.to(device)
        optimizer.zero_grad()

        images_emb = clr_model(images)
        augmented_images_emb = clr_model(augmented_images)
        loss = criterion(images_emb, augmented_images_emb)

        loss.backward()

        optimizer.step()
        tot_loss += loss.item()
        batch_tot_loss += loss.item()
        if (batch_idx % 200 == 1):
            print('batch loss=', batch_tot_loss)
    scheduler.step()



    # Validation
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(dataloaders['eval']):
            val_batch_loss = 0
            if (batch_num % 50 == 1):
                print("in batch", batch_num)
            batch_num=batch_num+1
            images, augmented_images = images[0], images[1]
            images, augmented_images = images.to(device), augmented_images.to(device)


            images_emb = clr_model(images)
            augmented_images_emb = clr_model(augmented_images)
            loss = criterion(images_emb, augmented_images_emb)



            val_loss += loss.item()
            val_batch_loss += loss.item()
        if (batch_idx % 200 == 1):
          print('batch loss=', val_batch_loss)

    print(f"epoch {epoch}train loss= {tot_loss/len(dataloaders['train'])}")
    print(f"epoch {epoch}val loss= {val_loss/len(dataloaders['train'])}")

    if epoch % 10 ==0:
        save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
        if not os.path.isdir(save_dir):
          os.mkdir(save_dir)
        modelstate_save_name = f'modelstate_bs_restart_r_{Config.spurious_correlation_strength}_LARGE_epoch{epoch}.pt'
        path_modelstate = os.path.join(save_dir, modelstate_save_name)
        torch.save(clr_model.state_dict(), path_modelstate)

        parameters_end = {
            'net': clr_model.state_dict(),
            'epoch': epoch,
            'loss': tot_loss
        }
        paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
        path_paramsend = os.path.join(save_dir, paramsend_save_name)
        torch.save(parameters_end, path_paramsend)


In [16]:
start_epoch=1
max_epoch=30

for epoch in range(start_epoch, start_epoch+max_epoch):
  train(epoch)


Epoch: 1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 6.238104820251465
in batch 51
in batch 101
in batch 151
in batch 201
epoch 1train loss= 5.632774736018891
epoch 1val loss= 1.1631027713735054

Epoch: 2


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 5.293218612670898
in batch 51
in batch 101
in batch 151
in batch 201
epoch 2train loss= 5.240399817202953
epoch 2val loss= 1.1520351754858138

Epoch: 3


...

in batch 1
batch loss= 4.973466873168945
in batch 51
in batch 101
in batch 151
in batch 201
epoch 28train loss= 4.941090223637033
epoch 28val loss= 1.056873268269478

Epoch: 29


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.920978546142578
in batch 51
in batch 101
in batch 151
in batch 201
epoch 29train loss= 4.933529270456193
epoch 29val loss= 1.0752566583613132

Epoch: 30


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.8997673988342285
in batch 51
in batch 101
in batch 151
in batch 201
epoch 30train loss= 4.932489352023348
epoch 30val loss= 1.0599915981292725


In [17]:
## Load the pre-trained model parameters
paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
path_paramsend = path_paramsend = os.path.join(save_dir, paramsend_save_name)
checkpoint = torch.load(path_paramsend, map_location = torch.device(device))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

start_epoch=epoch+1
clr_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

## PCA and 2D Scatter Plot

In [18]:
torch.cuda.empty_cache()

In [19]:
## https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA


torch.cuda.empty_cache()
device = 'cpu'
clr_model.to(device)

embeddings = []
targets = []

for batch_idx, ((images, _), labels) in enumerate(dataloaders['eval']):
  embeddings.append(clr_model(images).detach().cpu())
  targets.append(labels)
  torch.cuda.empty_cache()


# clr_model = None

embeddings = torch.cat(embeddings).detach().cpu().numpy()
targets = torch.cat(targets).cpu().numpy()

In [20]:
import pandas as pd
comp = 100
pca_2d = PCA(n_components=comp)
PCs_2d = pd.DataFrame(pca_2d.fit_transform(embeddings))
PCs_2d.columns = [f"PC{i+ 1}" for i in range(comp)]
pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')

In [21]:
pce = pca_2d.fit_transform(embeddings)

In [22]:
pca_2d.singular_values_

array([539.3439   , 505.32455  , 485.9453   , 400.70737  , 331.4959   ,
       322.91336  , 241.49309  , 204.24548  ,  99.13317  ,  61.06325  ,
        17.15247  ,  11.830786 ,  10.319187 ,   8.744855 ,   8.693472 ,
         8.441873 ,   8.043579 ,   7.54817  ,   7.031075 ,   6.877513 ,
         6.518043 ,   6.3597007,   6.1660886,   6.0534077,   5.7873545,
         5.6080284,   5.5624714,   5.4328485,   5.3223405,   5.1995783,
         5.065706 ,   4.8859763,   4.7908525,   4.699443 ,   4.5428057,
         4.4448895,   4.3831987,   4.364173 ,   4.317516 ,   4.2599444,
         4.095232 ,   3.9694543,   3.8336017,   3.759197 ,   3.686648 ,
         3.6336   ,   3.508201 ,   3.4614985,   3.3618846,   3.3416712,
         3.1920168,   3.1795928,   3.1201825,   3.0833118,   3.0270033,
         2.946208 ,   2.927271 ,   2.891732 ,   2.7772062,   2.7419875,
         2.6840045,   2.6270316,   2.5771232,   2.4976513,   2.409705 ,
         2.374697 ,   2.3427045,   2.3012512,   2.2501898,   2.1

In [23]:
pca_embs['true_label'] = targets

In [24]:
clusters = {k : np.stack([PCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [25]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

traces = []

for (k, v) in sorted(clusters.items()):
  traces.append(go.Scatter(
                    x = v[:, 2],
                    y = v[:, 3],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))
data = traces

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [26]:
for (true, spur), idxs in valset.group_partition.items():
  for i in idxs:
    pca_embs.loc[i, 'spurious_label'] = spur
pca_embs['spurious_label'] = pca_embs['spurious_label'].astype(int)

In [27]:
valset.group_partition.keys()

dict_keys([(3, 4), (1, 1), (0, 0), (0, 4), (2, 2), (2, 3), (4, 1), (2, 1), (4, 3), (0, 2), (3, 2), (0, 3), (2, 0), (3, 0), (1, 0), (4, 4), (3, 3), (2, 4), (1, 2), (1, 3), (1, 4), (3, 1), (4, 0), (0, 1), (4, 2)])

In [28]:
true_label_clusters = {
    # True label: {group, cluster_dim}
}
for l in pca_embs['true_label'].unique():
  true_cluster = pca_embs[pca_embs['true_label'] == l]
  true_emb = true_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,true_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  true_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [29]:
spurious_label_clusters = {
    # Spurious label: {group, cluster_dim}
}
for l in pca_embs['spurious_label'].unique():
  spurious_cluster = pca_embs[pca_embs['spurious_label'] == l]
  spurious_emb = spurious_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,spurious_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  spurious_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [30]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in true_label_clusters.items():
  traces = []
  for s in np.sort(v['spurious_label'].unique()):
    group = v[v['spurious_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC2_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"Subgroup {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - True Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

In [31]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in spurious_label_clusters.items():
  traces = []
  for s in np.sort(v['true_label'].unique()):
    group = v[v['true_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC1_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"True Label {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - Spurious Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_Spurious_Label_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

## t-SNE

In [32]:
from sklearn.manifold import TSNE

perplexity = 50

#T-SNE with two dimensions
tsne_2d = TSNE(n_components=2, perplexity=perplexity)

TCs_2d = pd.DataFrame(tsne_2d.fit_transform(embeddings))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
tsne_embs = pd.concat([pd.DataFrame(embeddings),TCs_2d], axis=1, join='inner')

In [33]:
tsne_embs

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,TC1_2d,TC2_2d
0,-1.991375,0.781524,-0.161806,0.273514,-0.251852,-0.230545,0.794584,-0.322194,-0.501190,-1.573455,...,0.244350,-0.490102,0.825796,-1.116243,-0.277615,1.138080,-2.157657,0.004434,-56.479057,-0.517250
1,-0.247669,0.617311,-0.157292,-0.367008,1.966240,0.616824,1.033686,-0.326313,0.379425,1.327853,...,-0.397538,1.892616,-1.443258,1.484512,-1.412699,-0.847493,1.121393,0.606100,9.683156,-4.246382
2,1.028928,0.879157,-1.378110,-0.811411,-0.698581,-1.111285,-2.736299,2.967212,-1.139339,-0.242599,...,2.858773,0.989423,-0.853248,0.665071,-0.006976,0.586573,-0.709012,-0.821270,8.315903,79.259850
3,-1.503344,0.618053,0.470094,0.654998,-0.091459,-1.967980,-0.297621,0.414168,0.314038,-0.926873,...,0.407674,-0.262510,0.544055,-0.603438,-0.203189,0.599773,-0.210550,-1.211465,-11.218361,-1.228181
4,0.372946,1.342141,-1.288977,0.739686,-0.470824,0.674006,-0.688949,-0.345195,0.259626,0.756421,...,-0.867437,-0.342489,0.528855,0.813171,1.424517,-1.394960,0.125830,-1.238745,56.226528,17.090654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-1.598268,0.174057,0.628648,0.186902,-0.102105,-0.448161,0.414997,0.470246,1.012841,-1.298756,...,0.738045,-0.600571,1.030470,-1.125106,0.086135,1.064640,-0.883095,-0.851551,-29.836349,-1.660297
9996,-1.262657,-0.130685,0.568599,-0.129027,-0.152376,-0.347540,0.423625,0.276767,0.670752,-1.000778,...,0.858875,-0.668203,0.990635,-0.997795,0.114512,0.842715,-0.804912,-0.437820,-33.678600,-10.753481
9997,0.847935,0.367826,-1.133335,-0.767992,1.444354,2.029223,1.627535,1.214289,0.660693,0.733190,...,-0.699602,2.094505,-0.992141,1.197552,-0.493736,-0.588530,0.437112,-0.898538,27.983715,46.831421
9998,0.203906,0.185908,-0.289098,-0.681369,1.685825,1.117029,1.386444,-0.435018,0.417054,0.988465,...,-1.071684,1.904822,-1.301955,1.237532,-1.047979,-0.629258,0.857787,0.116633,10.866878,12.071891


In [34]:
tsne_clusters = {k : np.stack([TCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [35]:
tsne_traces = []

for (k, v) in sorted(tsne_clusters.items()):
  tsne_traces.append(go.Scatter(
                    x = v[:, 0],
                    y = v[:, 1],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))

In [36]:
data = tsne_traces

title = "Visualizing Clusters in Two Dimensions Using t-SNE"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)